# Deploy Container Image
| This guide explains how to register a custom container image pushed to ECR as a Sagemaker Endpoint.


## Overview
Registering a custom container image as a Sagemaker Endpoint consists of three steps:

1. Register Sagemaker Model
2. Register Sagemaker Endpoint Config
3. Register Sagemaker Endpoint

In [ ]:
import boto3
client = boto3.client('sagemaker')

In [ ]:
# Create Sagemaker Model
create_model_response = client.create_model(
    ModelName='<SAGEMAKER_MODEL_NAME>', # ex) sm-dummy-inference-model
    PrimaryContainer={ #
        'Image': '<ECR_REPO_URI>', # ex) 9803771.dkr.ecr.ap-northeast-2.amazonaws.com/dummy-inference-model',
        'Environment': { }
    },
    ExecutionRoleArn='<IAM_ROLE_AmazonSagemakerFullAccess>' # ex) arn:aws:iam::980377:role/service-role/AmazonSageMaker-ExecutionRole-20231127T170565'
)

print("create model response:", create_model_response)

In [ ]:
# Create Sagemaker Endpoint Config
create_endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName='<ENDPOINT_CONFIG_NAME>', # ex) dummy-inference-serve-config',
    ProductionVariants=[
        {
            'ModelName': '<SAGEMAKER_MODEL_NAME>', # ex) sm-dummy-inference-model
            'VariantName': '<VARIENT_NAME>', # ex) variant-sm-dummy-inference-model-1
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.m4.xlarge'
        }
    ]
)

print('create endpoint config response:', create_endpoint_config_response)

In [ ]:
# Create Sagemaker Endpoint
create_endpoint_response = client.create_endpoint(
    EndpointName='<ENDPOINT_NAME>', # ex) dummy-inference-serve',
    EndpointConfigName='<ENDPOINT_CONFIG_NAME>', # ex) dummy-inference-serve-config'
)

print('create endpoint response: ', create_endpoint_response)

In [ ]:
# Calling Sagemaker Endpoint
sagemaker_runtime = boto3.client("runtime.sagemaker")
inference_response = sagemaker_runtime.invoke_endpoint(
    EndpointName='<ENDPOINT_NAME>',
    Body="Hello World"
)

response_str = inference_response["Body"].read().decode()
print(response_str)